In [1]:
#imports
import os
import random
import time
import torch
import torchvision
import random
import splitfolders
import numpy as np
from PIL import Image
import torch.nn as nn
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torch import optim
from torch.optim import lr_scheduler
from torchvision import datasets, transforms, models
from torch.utils.tensorboard import SummaryWriter 
from torch.utils.data.sampler import SubsetRandomSampler
import imagefolder
torch.backends.cudnn.benchmark = True

In [2]:
def set_random_seed(seed):
    """Set random seed.

    Args:
        seed (int): Seed to be used.
        deterministic (bool): Whether to set the deterministic option for
            CUDNN backend, i.e., set `torch.backends.cudnn.deterministic`
            to True and `torch.backends.cudnn.benchmark` to False.
            Default: False.
    """
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    
set_random_seed(1234)

In [3]:
root_folder = 'C:\\Users\\Oge\\Documents\\Work\\Official\\hierachical_classifier\\data\\val' 
imgfoldermap = imagefolder.HeirarchicalLabelMap(root_folder, level_names=['category', 'subcategory', 'item'])

In [4]:
imgfoldermap.leaf_class_labels

{'Golden-Delicious': 0,
 'Granny-Smith': 1,
 'Pink-Lady': 2,
 'Royal-Gala': 3,
 'Cantaloupe': 4,
 'Galia-Melon': 5,
 'Honeydew-Melon': 6,
 'Watermelon': 7,
 'Conference': 8,
 'Bravo-Apple-Juice': 9,
 'Bravo-Orange-Juice': 10,
 'God-Morgon-Apple-Juice': 11,
 'God-Morgon-Orange-Juice': 12,
 'God-Morgon-Orange-Red-Grapefruit-Juice': 13,
 'God-Morgon-Red-Grapefruit-Juice': 14,
 'Arla-Ecological-Medium-Fat-Milk': 15,
 'Arla-Medium-Fat-Milk': 16,
 'Arla-Standard-Milk': 17,
 'Garant-Ecological-Medium-Fat-Milk': 18,
 'Garant-Ecological-Standard-Milk': 19,
 'Oatly-Oat-Milk': 20,
 'Oatly-Natural-Oatghurt': 21,
 'Arla-Sour-Cream': 22,
 'Alpro-Blueberry-Soyghurt': 23,
 'Alpro-Vanilla-Soyghurt': 24,
 'Arla-Mild-Vanilla-Yoghurt': 25,
 'Valio-Vanilla-Yoghurt': 26,
 'Yoggi-Strawberry-Yoghurt': 27,
 'Yoggi-Vanilla-Yoghurt': 28,
 'Yellow-Onion': 29,
 'Orange-Bell-Pepper': 30,
 'Red-Bell-Pepper': 31,
 'Yellow-Bell-Pepper': 32,
 'Floury-Potato': 33,
 'Sweet-Potato': 34,
 'Beef-Tomato': 35,
 'Vine-Tomato':

In [5]:
from torch.utils.data import Dataset
class HierarchicalDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, root_dir, transform=None, labelmap=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.leaf_paths = labelmap.get_leaf_paths()
        self.root_dir = root_dir
        self.transform = transform
        self.labelmap = labelmap
        self.dataset = self.make_dataset()
        
        
    def make_dataset(self, extensions = ['.jpg', '.jpeg'], is_valid_file = None):
        """Generates a list of samples of a form (path_to_sample, class).

        See :class:`DatasetFolder` for details.

        """
        class_to_idx = self.labelmap.leaf_class_labels

        '''
        both_none = extensions is None and is_valid_file is None
        both_something = extensions is not None and is_valid_file is not None
        if both_none or both_something:
            raise ValueError("Both extensions and is_valid_file cannot be None or not None at the same time")

        if extensions is not None:
            def is_valid_file(x: str) -> bool:
                return has_file_allowed_extension(x, cast(Tuple[str, ...], extensions))

        is_valid_file = cast(Callable[[str], bool], is_valid_file)
        '''

        instances = []
        available_classes = set()
        for leaf_path in self.leaf_paths:
            if not os.path.isdir(leaf_path):
                continue
            target_class = os.path.basename(leaf_path)
            class_index = class_to_idx[target_class]

            for root, _, filenames in sorted(os.walk(leaf_path, followlinks=False)):
                for fname in sorted(filenames):
                    path = os.path.join(root, fname)
                    extension = (os.path.splitext(fname)[1]).lower().strip()
                    #TODO redo to is valid file
                    if os.path.isfile(path) and extension in extensions:
                        item = path, class_index
                        instances.append(item)

                        if target_class not in available_classes:
                            available_classes.add(target_class)

        empty_classes = set(class_to_idx.keys()) - available_classes

        if empty_classes:
            msg = f"Found not valid file for the classes {', '.join(sorted(empty_classes))}. "
            if extensions is not None:
                msg += f"Supported extensions are: {', '.join(extensions)}"
            raise FileNotFoundError(msg)

        return np.array(instances)


    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        img_name, indices = self.dataset[idx]
        image = Image.open(img_name)
        
        if self.transform:
            image = self.transform(image)
        
        #sample = {'image': image, 'label': indices}

        return image, int(indices)

In [6]:
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

tr_transform = transforms.Compose([transforms.RandomResizedCrop(224), transforms.RandomHorizontalFlip(), transforms.ToTensor(), transforms.Normalize(mean, std)])

begin = time.time()
tr_dataset = HierarchicalDataset(root_folder, labelmap=imgfoldermap, transform=tr_transform)
end = time.time() - begin

In [7]:
dataset_size = len(tr_dataset)

In [8]:
a, b= tr_dataset[5]
a.shape,b

(torch.Size([3, 224, 224]), 0)

In [9]:
batchsize = 8
workers = 0
pinmemory = True
train_loader = torch.utils.data.DataLoader(tr_dataset, batch_size=batchsize, num_workers=workers, shuffle=True, pin_memory=pinmemory)

In [10]:
outsize = 512

class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(44944, 2048)  # 5*5 from image dimension
        self.fc2 = nn.Linear(2048, 1024)
        self.fc3 = nn.Linear(1024, outsize)
        self.hsoftmax = imagefolder.HierarchicalSoftmax(labelmap=imgfoldermap, input_size=outsize, level_weights=None)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square, you can specify with a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = torch.flatten(x, 1) # flatten all dimensions except the batch dimension
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        x = self.hsoftmax(x)
        return x


model = Net()
print(model)

ModuleDict(
  (root): Linear(in_features=512, out_features=3, bias=True)
  (category_Fruit_1): Linear(in_features=512, out_features=3, bias=True)
  (category_Packages_1): Linear(in_features=512, out_features=7, bias=True)
  (category_Vegetables_1): Linear(in_features=512, out_features=4, bias=True)
  (subcategory_Apple_2): Linear(in_features=512, out_features=4, bias=True)
  (subcategory_Melon_2): Linear(in_features=512, out_features=4, bias=True)
  (subcategory_Pear_2): Linear(in_features=512, out_features=1, bias=True)
  (subcategory_Juice_2): Linear(in_features=512, out_features=6, bias=True)
  (subcategory_Milk_2): Linear(in_features=512, out_features=5, bias=True)
  (subcategory_Oat-Milk_2): Linear(in_features=512, out_features=1, bias=True)
  (subcategory_Oatghurt_2): Linear(in_features=512, out_features=1, bias=True)
  (subcategory_Sour-Cream_2): Linear(in_features=512, out_features=1, bias=True)
  (subcategory_Soyghurt_2): Linear(in_features=512, out_features=2, bias=True)
  (s

In [11]:
optimizer = optim.Adam(model.parameters())

# Decay LR by a factor of 0.1 every 10 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

#set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

epochs = 1000

criterion = criterion = torch.nn.NLLLoss() # nn.CrossEntropyLoss()

model = model.to(device)

In [12]:
print(device)

cuda


In [13]:
len(train_loader)

24

In [15]:
for e in range(epochs):
    
    running_loss = 0.0
    running_correct = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = model(inputs)
        outputs = outputs[1]
        _, preds = torch.max(outputs, 1)
        #class_probs_batch = [F.softmax(output, dim=0) for output in outputs]
        #outputs = F.log_softmax(outputs, dim=0)
        
        loss = criterion(outputs, labels)

        # zero the parameter gradients
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        running_correct += torch.sum(preds == labels)
        
    epoch_loss = running_loss / dataset_size
    epoch_accuracy = running_correct.double() / dataset_size
        
    print(f'Loss: {epoch_loss:.4f}; Accuracy: {epoch_accuracy:.4f}')
    print('')
    


Loss: 3.9694; Accuracy: 0.0486

Loss: 3.2318; Accuracy: 0.0541

Loss: 2.9906; Accuracy: 0.0811

Loss: 3.0086; Accuracy: 0.0919

Loss: 2.9368; Accuracy: 0.0649

Loss: 2.7165; Accuracy: 0.1405

Loss: 2.5776; Accuracy: 0.1784

Loss: 2.6657; Accuracy: 0.1189

Loss: 2.5021; Accuracy: 0.2054

Loss: 2.3053; Accuracy: 0.2757

Loss: 2.8759; Accuracy: 0.1838

Loss: 2.9655; Accuracy: 0.1297

Loss: 2.6669; Accuracy: 0.1676

Loss: 2.4890; Accuracy: 0.2108

Loss: 2.5459; Accuracy: 0.2162

Loss: 2.4466; Accuracy: 0.2541

Loss: 2.3121; Accuracy: 0.2162

Loss: 2.1869; Accuracy: 0.2486

Loss: 2.3627; Accuracy: 0.2541

Loss: 2.1054; Accuracy: 0.3297

Loss: 2.1522; Accuracy: 0.3189

Loss: 2.3924; Accuracy: 0.2432

Loss: 2.1332; Accuracy: 0.3459

Loss: 2.1435; Accuracy: 0.3459

Loss: 2.0042; Accuracy: 0.3351

Loss: 1.8869; Accuracy: 0.4432

Loss: 1.9528; Accuracy: 0.4162

Loss: 1.9177; Accuracy: 0.4378

Loss: 2.3964; Accuracy: 0.3730

Loss: 1.6884; Accuracy: 0.4432

Loss: 2.0604; Accuracy: 0.3676

Loss: 2.

In [16]:
inputs, labels = next(iter(train_loader))

In [18]:
outputs = model(inputs.to(device))

In [19]:
logits = torch.exp(outputs[0])
#print(logits)
h = imgfoldermap.get_heirarchies(logits)
#print(h)
for l in h:
    print(imgfoldermap.get_heirarchy_label(l))

['Fruit', 'Apple', 'Pink-Lady']
['Packages', 'Juice', 'God-Morgon-Orange-Juice']
['Packages', 'Oatghurt', 'Oatly-Natural-Oatghurt']
['Packages', 'Juice', 'God-Morgon-Orange-Juice']
['Packages', 'Yoghurt', 'Yoggi-Strawberry-Yoghurt']
['Packages', 'Milk', 'Garant-Ecological-Medium-Fat-Milk']
['Packages', 'Milk', 'Arla-Ecological-Medium-Fat-Milk']
['Packages', 'Milk', 'Arla-Medium-Fat-Milk']


In [20]:
labels

tensor([ 2, 10, 21, 12, 27, 18, 15, 15])

In [21]:
imgfoldermap.leaf_class_labels

{'Golden-Delicious': 0,
 'Granny-Smith': 1,
 'Pink-Lady': 2,
 'Royal-Gala': 3,
 'Cantaloupe': 4,
 'Galia-Melon': 5,
 'Honeydew-Melon': 6,
 'Watermelon': 7,
 'Conference': 8,
 'Bravo-Apple-Juice': 9,
 'Bravo-Orange-Juice': 10,
 'God-Morgon-Apple-Juice': 11,
 'God-Morgon-Orange-Juice': 12,
 'God-Morgon-Orange-Red-Grapefruit-Juice': 13,
 'God-Morgon-Red-Grapefruit-Juice': 14,
 'Arla-Ecological-Medium-Fat-Milk': 15,
 'Arla-Medium-Fat-Milk': 16,
 'Arla-Standard-Milk': 17,
 'Garant-Ecological-Medium-Fat-Milk': 18,
 'Garant-Ecological-Standard-Milk': 19,
 'Oatly-Oat-Milk': 20,
 'Oatly-Natural-Oatghurt': 21,
 'Arla-Sour-Cream': 22,
 'Alpro-Blueberry-Soyghurt': 23,
 'Alpro-Vanilla-Soyghurt': 24,
 'Arla-Mild-Vanilla-Yoghurt': 25,
 'Valio-Vanilla-Yoghurt': 26,
 'Yoggi-Strawberry-Yoghurt': 27,
 'Yoggi-Vanilla-Yoghurt': 28,
 'Yellow-Onion': 29,
 'Orange-Bell-Pepper': 30,
 'Red-Bell-Pepper': 31,
 'Yellow-Bell-Pepper': 32,
 'Floury-Potato': 33,
 'Sweet-Potato': 34,
 'Beef-Tomato': 35,
 'Vine-Tomato':